<a href="https://colab.research.google.com/github/itsSrijan/whatsapp-chat-analysis/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 7.9 MB/s 


In [3]:
import os
import copy
import itertools
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, LSTM
from tensorflow.keras.layers import BatchNormalization, Activation, GlobalMaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score
from keras_tuner.tuners import RandomSearch, Hyperband
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import RandomizedSearchCV
# from scikeras.wrappers import KerasClassifier
# from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, classification_report

mpl.rcParams['figure.figsize'] = (12,8)
mpl.rcParams['axes.grid'] = False

Can be provided separetely

* Inputs
1. no. of samples in each frame = ns
2. joints/accelerometer which onw wants to analyse = j
3. filters = f1, f2
4. kernel_size = k
5. strides = s
6. learning_rate = lr
7. validation_split = vs
8. batch_size = b
9. epochs = e  

* Functions
1. takes **ns, j, (directory)** and returns **scaled_train, train_label, scaled_test, test_label**
2. takes 
3. takes **f, k, lr, s** and returns **model**
 

In [4]:
#Inputs

directory = '/content/drive/MyDrive/Dataset'
sampling_freq = 1024
duration = 256
nojoints = 30
nosamples = 64 #nosamples = [64, 128, 256, 512]
timesteps = 8
# strides = 1
# learning_rate = 0.001
# batch_size = 32
epochs = 100
# validation_split = 0.1

frames = (sampling_freq * duration)/(nosamples * timesteps)

In [5]:
frames

512.0

In [6]:
#here, we are reading the different .csv from the directory which contain the data

i = 0
DF = {} #this will contain the damaged frames for all joint
UFN = {} #this will contain undamaged shuffled frames for all joints joint
for filename in os.scandir(directory):
     if filename.is_file():
        i = i + 1 
        df = pd.read_csv(filename.path)
        if str(i) in filename.name:
            col = 'J' + str(i)
            key = 'DF'+ str(i)
            DF[key] = df[col] 
        for j in range(1, nojoints + 1):
            if j!=i:
              col = 'J' + str(j)
              key_UFN = 'UFN' + str(j)
              if key_UFN not in UFN:
                UFN[key_UFN] = []
              UFN[key_UFN].append(df[col])

for keys in DF:
  temp1 = np.array(DF[keys])
  temp2 = np.array(np.split(temp1,sampling_freq * duration / nosamples)) #splitting damaged dataset into frames
  DF[keys] = np.array(np.split(temp2, frames))

for keys in UFN:
  temp1 = np.array(UFN[keys])
  temp2 = np.reshape(temp1, temp1.shape[0] * temp1.shape[1])
  temp3 = np.array(np.split(temp2, sampling_freq * duration * nojoints / nosamples)) #splitting undamaged dataset into frames, each having 128 samples
  temp4 = np.array(np.split(temp3, temp3.shape[0] / timesteps))
  np.random.shuffle(temp4) #shuffling the undamaged frames
  UFN[keys] = temp4   #[:int(frames),:] #selecting first 'n' frames equal to no. of damaged frames

In [7]:
y1 = copy.deepcopy(UFN['UFN1'])
y2 = copy.deepcopy(DF['DF1'])


y1_label = np.zeros((y1.shape[0], 1))
y2_label = np.ones((y2.shape[0], 1))
# one_hot_label = to_categorical(input_labels)

# now, y1 contains 4096 undamaged frames and y2 contains 4096 damaged frames. In the paper, half of the frames from both undamaged and 
# damaged was used for training. 

# we will take 10% of from half of damaged and half of undamaged frames to create a validation set

 


y1_half = y1[:int(frames/2)]
y2_half = y2[:int(frames/2)]
y1_label_half = y1_label[:int(frames/2)]
y2_label_half = y2_label[:int(frames/2)]
train = np.concatenate((y1_half, y2_half), axis = 0)
train_label = np.concatenate((y1_label_half, y2_label_half), axis = 0)
train_label = np.reshape(train_label, (train_label.shape[0], 1))
train_label = to_categorical(train_label)
train, train_label = shuffle (train, train_label, random_state = 42)

# Here, we are using the remaining half for test purpose

y1_rem = y1[int(frames/2):int(frames)]
y2_rem = y2[int(frames/2):int(frames)]
y1_label_rem = y1_label[int(frames/2):int(frames)]
y2_label_rem = y2_label[int(frames/2):int(frames)]
test = np.concatenate((y1_rem, y2_rem), axis = 0)
test_label = np.concatenate((y1_label_rem, y2_label_rem), axis = 0)
test_label = np.reshape(test_label, (test_label.shape[0], 1))
test_label = to_categorical(test_label)
test, test_label = shuffle(test, test_label, random_state = 42)


In [8]:
# scaling the training and test data

scaler = StandardScaler()
scaled_train = scaler.fit_transform(train.reshape(-1, train.shape[-1])).reshape(train.shape)
scaled_test = scaler.transform(test.reshape(-1, test.shape[-1])).reshape(test.shape)

In [9]:
# train_generator = TimeseriesGenerator(scaled_train, train_label, length = timesteps, stride = timesteps, batch_size = 1,sampling_rate = 1)
# test_generator = TimeseriesGenerator(scaled_test, test_label, length = timesteps, stride = timesteps, batch_size = 1, sampling_rate = 1)

In [10]:
# def LSTM_classifier():

#   model = Sequential()
#   model.add(LSTM(units = 64 , activation='tanh', input_shape=(timesteps, nosamples), return_sequences=True))
#   model.add(Dropout(rate = 0.2))
#   model.add(LSTM(units = 128, activation='tanh', return_sequences=True))
#   model.add(Dropout(rate = 0.2))
#   model.add(LSTM(units = 64, activation='tanh', return_sequences=False))
#   model.add(Dropout(rate = 0.2))
#   model.add(Flatten())
#   model.add(Dense(units = 32))
#   model.add(Dropout(rate = 0.2))
#   model.add(Dense(units = 2, activation = 'softmax'))
#   model.compile(optimizer = Adam(learning_rate = learning_rate), loss = 'binary_crossentropy', metrics = ['accuracy'])
#   return model



In [11]:
# # timesteps = [2, 4, 8, 16] timesteps = timesteps,
# # nolayers = [1, 2, 3, 4, 5, 6] nolayers = nolayers
# units = [1024, 512, 256, 128, 64, 32]
# rate = [0.2, 0.5]
# learning_rate = [0.0003, 0.001, 0.01]
# batch_size = [64, 256, 512]
# nodes = (
#     list(itertools.product(units, repeat=1))
#     + list(itertools.product(units, repeat=2))
#     + list(itertools.product(units, repeat=3))
#     + list(itertools.product(units, repeat=4))
#     + list(itertools.product(units, repeat=5))
#     + list(itertools.product(units, repeat=6))
# )

# param_grid = dict(  nodes = nodes, rate = rate, learning_rate = learning_rate, batch_size = batch_size, units = units)

In [12]:
# def build_model(hp):
#   model = keras.Sequential()
#   model.add(InputLayer(input_shape = (8,64)))
#   for i in range(hp.Int('layers', 1, 6)): 
#       model.add(LSTM(units = hp.Int('units' + str(i+1), 32, 1024, 32), activation = 'tanh', return_sequences = True, dropout = hp.Choice('rate' + str(i+1), [0.2, 0.5], ordered = False)))
#   model.add(Flatten())
#   model.add(Dense(units = hp.Choice('units', [16, 32, 64], ordered = False)))
#   model.add(Dropout(rate = hp.Choice('rate', [0.2, 0.5], ordered = False)))
#   model.add(Dense(units = 2, activation='softmax'))

#   model.compile(optimizer = Adam(learning_rate = hp.Choice('learning_rate', [0.0003, 0.001, 0.01], ordered = False)), loss='binary_crossentropy', metrics = ['accuracy'])
#   return model


In [50]:
class MyHyperModel(kt.HyperModel):
  def build(self, hp):
    model = keras.Sequential()
    model.add(InputLayer(input_shape = (8,64)))
    for i in range(hp.Int('layers', 1, 6)): 
      model.add(LSTM(units = hp.Choice('units' + str(i+1),  [32, 64, 128, 512, 1024], ordered = False, default = 32), activation = 'tanh', return_sequences = True))
      model.add(Dropout(rate = hp.Float('rate' + str(i+1), 0.1, 0.2, 0.05)))
    model.add(Flatten())
    # model.add(Dropout(rate = hp.Float('rate', 0.1, 0.2, 0.05 )))
    model.add(Dense(units = hp.Choice('units', [8, 16, 32, 64], ordered = False)))
    
    model.add(Dense(units = 2, activation='softmax'))

    model.compile(optimizer = Adam(learning_rate = hp.Choice('learning_rate', [0.0003, 0.001, 0.01], ordered = False)), loss='binary_crossentropy', metrics = ['accuracy'])
    return model

  def fit_it(self, hp, model, x, y, validation_split, callbacks=None, *args, **kwargs):
    return model.fit(x, y, validation_split, batch_size = hp.Choice('batch_size', [64, 256, 512], ordered = False), verbose = 2, *args, **kwargs,)

In [56]:
tuner = RandomSearch(
    MyHyperModel(), 
    objective ='val_accuracy', 
    max_trials = 20, 
    executions_per_trial = 3,
    directory = '/content/drive/MyDrive/Colab Notebooks', 
    project_name = 'tuner12_with_callback',)

In [57]:
early_stopping = EarlyStopping(monitor='val_accuracy', 
    patience=2, 
    min_delta=0.005, 
    mode='max')

In [58]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 6, 'step': 1, 'sampling': None}
units1 (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64, 128, 512, 1024], 'ordered': False}
rate1 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.2, 'step': 0.05, 'sampling': None}
units (Choice)
{'default': 8, 'conditions': [], 'values': [8, 16, 32, 64], 'ordered': False}
learning_rate (Choice)
{'default': 0.0003, 'conditions': [], 'values': [0.0003, 0.001, 0.01], 'ordered': False}


In [59]:
tuner.search(scaled_train, train_label, validation_split = 0.2, epochs = 30, callbacks = [early_stopping])

Trial 20 Complete [00h 00m 39s]
val_accuracy: 0.9352750778198242

Best val_accuracy So Far: 0.9805825153986613
Total elapsed time: 00h 11m 14s
INFO:tensorflow:Oracle triggered exit


In [60]:
tuner.results_summary()

Results summary
Results in /content/drive/MyDrive/Colab Notebooks/tuner12_with_callback
Showing 10 best trials
Trial summary
Hyperparameters:
layers: 2
units1: 128
rate1: 0.1
units: 8
learning_rate: 0.01
units2: 32
rate2: 0.1
units3: 1024
rate3: 0.15000000000000002
units4: 64
rate4: 0.20000000000000004
units5: 64
rate5: 0.1
Score: 0.9805825153986613
Trial summary
Hyperparameters:
layers: 2
units1: 32
rate1: 0.20000000000000004
units: 16
learning_rate: 0.01
units2: 32
rate2: 0.1
Score: 0.9773462812105814
Trial summary
Hyperparameters:
layers: 1
units1: 32
rate1: 0.15000000000000002
units: 64
learning_rate: 0.001
units2: 128
rate2: 0.20000000000000004
units3: 128
rate3: 0.1
units4: 32
rate4: 0.20000000000000004
units5: 1024
rate5: 0.15000000000000002
Score: 0.9741100271542867
Trial summary
Hyperparameters:
layers: 1
units1: 1024
rate1: 0.20000000000000004
units: 64
learning_rate: 0.0003
units2: 512
rate2: 0.15000000000000002
units3: 512
rate3: 0.15000000000000002
units4: 128
rate4: 0.200

In [66]:
best_params = tuner.get_best_hyperparameters(num_trials = 1)[0]
best_params.values

{'layers': 2,
 'learning_rate': 0.01,
 'rate1': 0.1,
 'rate2': 0.1,
 'rate3': 0.15000000000000002,
 'rate4': 0.20000000000000004,
 'rate5': 0.1,
 'units': 8,
 'units1': 128,
 'units2': 32,
 'units3': 1024,
 'units4': 64,
 'units5': 64}

In [67]:
tuner.get_best_models(num_models = 1)[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 8, 128)            98816     
                                                                 
 dropout (Dropout)           (None, 8, 128)            0         
                                                                 
 lstm_1 (LSTM)               (None, 8, 32)             20608     
                                                                 
 dropout_1 (Dropout)         (None, 8, 32)             0         
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 8)                 2056      
                                                                 
 dense_1 (Dense)             (None, 2)                 1

In [69]:
model = tuner.hypermodel.build(best_params)
history = model.fit(scaled_train, train_label, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch))

Epoch 1/50
13/13 [==============================] - 4s 68ms/step - loss: 0.4508 - accuracy: 0.7946 - val_loss: 0.3675 - val_accuracy: 0.8447
Epoch 2/50
13/13 [==============================] - 0s 9ms/step - loss: 0.0932 - accuracy: 0.9780 - val_loss: 0.2537 - val_accuracy: 0.9126
Epoch 3/50
13/13 [==============================] - 0s 8ms/step - loss: 0.0967 - accuracy: 0.9731 - val_loss: 0.0803 - val_accuracy: 0.9612
Epoch 4/50
13/13 [==============================] - 0s 10ms/step - loss: 8.1378e-04 - accuracy: 1.0000 - val_loss: 0.1227 - val_accuracy: 0.9709
Epoch 5/50
13/13 [==============================] - 0s 9ms/step - loss: 3.3188e-04 - accuracy: 1.0000 - val_loss: 0.0913 - val_accuracy: 0.9709
Epoch 6/50
13/13 [==============================] - 0s 8ms/step - loss: 1.1432e-05 - accuracy: 1.0000 - val_loss: 0.0738 - val_accuracy: 0.9806
Epoch 7/50
13/13 [==============================] - 0s 9ms/step - loss: 7.7893e-06 - accuracy: 1.0000 - val_loss: 0.0711 - val_accuracy: 0.9806
Ep

In [70]:
hypermodel = tuner.hypermodel.build(best_params)

# Retrain the model
hypermodel.fit(scaled_train, train_label, epochs=best_epoch, validation_split=0.2)

Epoch 1/6
13/13 [==============================] - 4s 70ms/step - loss: 0.4011 - accuracy: 0.7995 - val_loss: 0.1036 - val_accuracy: 0.9612
Epoch 2/6
13/13 [==============================] - 0s 9ms/step - loss: 0.0678 - accuracy: 0.9804 - val_loss: 0.0668 - val_accuracy: 0.9806
Epoch 3/6
13/13 [==============================] - 0s 9ms/step - loss: 0.0212 - accuracy: 0.9951 - val_loss: 0.2511 - val_accuracy: 0.9417
Epoch 4/6
13/13 [==============================] - 0s 8ms/step - loss: 0.0278 - accuracy: 0.9927 - val_loss: 0.3897 - val_accuracy: 0.9515
Epoch 5/6
13/13 [==============================] - 0s 8ms/step - loss: 0.0314 - accuracy: 0.9902 - val_loss: 0.2734 - val_accuracy: 0.9515
Epoch 6/6
13/13 [==============================] - 0s 10ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.2306 - val_accuracy: 0.9417


In [31]:
class Employee:
  def __init__(self, pay):
    self.pay = pay

  def Max(self):
    max_pay = 2 * self.pay
    return max_pay

  def MaxMax(self):
    return Employee.Max(self) * 3

In [32]:
emp1 = Employee(5)
emp1.MaxMax()

30

In [ ]:
def layer_creation(model, nodes, rate, timesteps, nosamples):
  for x in tf.range(len(nodes)):
    if x == 0:
      model.add(LSTM(units = nodes[x], activation = 'tanh', input_shape=(timesteps, nosamples), return_sequences = True))
      model.add(Dropout(rate = rate))
    elif x == len(nodes) - 1:
      model.add(LSTM(units = nodes[x], activation = 'tanh', return_sequences = False))
      model.add(Dropout(rate = rate))
    else:
      model.add(LSTM(units = nodes[x], activation = 'tanh', return_sequences = True))
      model.add(Dropout(rate = rate))
  return model

In [ ]:
# def LSTM_classifier(nodes, units, rate, learning_rate, timesteps = 8, nosamples = 64):

#   model = Sequential()

#   layer_creation(model, nodes, rate, timesteps, nosamples)
  
#   model.add(Flatten())
  
#   model.add(Dense(units = units))
  
#   model.add(Dropout(rate = rate))

#   model.add(Dense(units = 2, activation = 'softmax'))

#   model.compile(optimizer = Adam(learning_rate = learning_rate), loss = 'binary_crossentropy', metrics = ['accuracy'])
  
#   return model

# model = KerasClassifier(model = LSTM_classifier, verbose = 0)

# model = KerasClassifier(model = LSTM_classifier, units = None, rate = None, nodes = None, learning_rate = None, batch_size = None, verbose = 0)

# def hypertuning_lstm(estimator, param_distributions, n_iter, cv, X, y):
#   rscv = RandomizedSearchCV(estimator = estimator, param_distributions = param_distributions, n_iter = n_iter, cv = cv, scoring = roc_auc_score(estimator, scaled_test, test_label))
#   rscv.fit(X, y)
#   ht_params = rscv.best_params_
#   ht_score = rscv.best_score_

#   return ht_params, ht_score

# lstm_parameters, lstm_score = hypertuning_lstm(estimator = model, param_distributions = param_grid, cv = 3, X = scaled_train, y = train_label, n_iter = 60)
# print(lstm_parameters, lstm_score)

  # model = Sequential()
  # model.add(LSTM(units = 64 , activation='tanh', input_shape=(timesteps, nosamples), return_sequences=True))
  # model.add(Dropout(rate = 0.2))
  # model.add(LSTM(units = 128, activation='tanh', return_sequences=True))
  # model.add(Dropout(rate = 0.2))
  # model.add(LSTM(units = 64, activation='tanh', return_sequences=False))
  # model.add(Dropout(rate = 0.2))
  # model.add(Flatten())
  # model.add(Dense(units = 32))
  # model.add(Dropout(rate = 0.2))
  # model.add(Dense(units = 2, activation = 'softmax'))
  # model.compile(optimizer = Adam(learning_rate = learning_rate), loss = 'binary_crossentropy', metrics = ['accuracy'])
  # return model

In [ ]:
# model = KerasClassifier(build_fn = LSTM_classifier, batch_size = batch_size, epochs = epochs)
# accuracies = cross_val_score(estimator = model, X = train, y = train_label, cv = KFold(3), n_jobs = -1)
# np.mean(accuracies)
# from sklearn.metrics import make_scorer
# from sklearn.metrics import accuracy_score, precision_score, recall_score
# score = make_scorer(accuracy_score)
# kfold_validation = KFold(3)
# results = cross_val_score(model, train, train_label, cv = kfold_validation, scoring = score)
# print(results)
# print(np.mean(results))

In [ ]:
model.fit(train, train_label, validation_split = 0.1,  batch_size = 64, epochs = epochs, verbose = 2, callbacks = [early_stopping])

In [ ]:
predictions = model.predict(test, verbose = 1)
predictions.shape

In [ ]:
# q = np.concatenate((test_label[1:256], test_label[2048:2048+256]), axis = 0)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(test_label.argmax(axis = 1), np.round(predictions).argmax(axis = 1)) #with threshhold = 0.5
plt.show()

We can use ROC curve to find the **threshhold** that will give maximum accuracy i.e when **TPR** (True Positive Rate) is almost equal to **TNR** (True Negative Rate)

With threshold = 0.5, **TPR** = 244/(244 + 12) i.e. **0.953** and **TNR** = 247/(247 + 8) i.e. **0.968**

In [ ]:
weights_binary = model.get_weights()

In [ ]:
# train_generator[0][0].shape

In [ ]:
# train_generator[510][0]

In [ ]:
predictions

In [ ]:
np.count_nonzero(test_label.argmax(axis = 1))